# Week 7 – "Next Hub + Housing Opportunity" Scoring


**Composite metrics:**
1. **Company (firm) growth rate** — new firm formation growth (e.g. firms_founded_yoy).
2. **Industry diversity** — less concentration = more diverse (e.g. 1 − HHI or industry_count).
3. **Housing price growth** — appreciation (e.g. FHFA YoY, Zillow YoY).
4. **Housing affordability** — relative affordability (e.g. lower price level or moderate growth = more "opportunity"; no income in data so we use price level and growth tradeoff).

**Deliverables:** Ranked metro lists (overall and by industry-correlated score), normalized composite, and discussion of tradeoffs.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path("..") / "data"
QFILE = DATA_DIR / "firmscape_integrated_cbsa_quarterly.csv"

In [2]:
# Load and restrict to recent period with good coverage (e.g. 2010+)
df = pd.read_csv(QFILE)
df["yq"] = pd.to_datetime(df["yq"].astype(str).str.replace("Q", "-") + "-01", format="%Y-%m-%d", errors="coerce")
df = df[df["year"] >= 2010].copy()

# Metro-level aggregates (one row per CBSA): mean over quarters for each metric
agg_cols = {
    "firms_founded_yoy": "mean",      # company growth rate
    "industry_count_new": "mean",     # diversity (more industries = more diverse)
    "top_industry_share_new": "mean", # concentration (1 - this = diversity)
    "fhfa_yoy": "mean",               # housing price growth
    "zillow_yoy": "mean",
    "fhfa_index": "mean",             # level (for affordability proxy)
    "zillow_price_q": "mean",
}
use = {k: v for k, v in agg_cols.items() if k in df.columns}
metro = df.groupby("cbsa_code").agg(use).reset_index()
# Metro names
names = df.groupby("cbsa_code")["metro_name"].first()
metro["metro_name"] = metro["cbsa_code"].map(names)
metro = metro.dropna(subset=list(use.keys()), how="all")
metro.head(10)

,cbsa_code,firms_founded_yoy,industry_count_new,top_industry_share_new,fhfa_yoy,zillow_yoy,fhfa_index,zillow_price_q,metro_name
0,10180,0.305556,1.500000,0.805556,0.047098,0.039104,231.281746,141566.713147,"Abilene, TX"
1,10420,0.330303,3.272727,0.421212,0.041928,0.039761,168.101270,146960.232456,"Akron, OH"
2,10500,0.666667,2.000000,0.666667,0.029597,0.026539,172.603333,112398.515152,"Albany, GA"
3,10540,0.000000,1.000000,1.000000,0.056916,0.058359,256.409206,248380.259206,"Albany, OR"
4,10580,NaN,NaN,NaN,0.036377,NaN,222.076349,NaN,"Albany-Schenectady-Troy, NY"
5,10740,0.030754,6.166667,0.263889,0.039687,0.041691,196.360635,218058.781745,"Albuquerque, NM"
6,10780,NaN,NaN,NaN,0.025289,0.023725,217.364286,144068.095299,"Alexandria, LA"
7,10900,NaN,NaN,NaN,0.039008,NaN,205.300476,NaN,"Allentown-Bethlehem-Easton, PA-NJ"
8,11020,-0.166667,1.000000,1.000000,0.035215,0.022094,207.499365,129100.146850,"Altoona, PA"
9,11100,0.234848,1.636364,0.780303,0.038917,0.035488,210.677619,148469.528442,"Amarillo, TX"


## Define and normalize composite growth signals

- **Company growth:** `firms_founded_yoy` (higher = more growth).
- **Industry diversity:** `industry_count_new` (higher = more diverse); alternatively `1 - top_industry_share_new`.
- **Housing price growth:** `fhfa_yoy` or `zillow_yoy` (higher = more appreciation).
- **Housing affordability:** No income data → use *inverse* of price level (lower Zillow price = more affordable) and optionally penalize very high price growth (so "opportunity" = still affordable or moderate growth). Here we use **negative** of normalized price level so higher score = more affordable.

Normalize each to [0, 1] or z-score so composites are comparable.

In [3]:
def normalize_0_1(series: pd.Series) -> pd.Series:
    """Min-max to [0, 1]. Handles NaN."""
    s = series.dropna()
    if s.empty or s.min() == s.max():
        return series
    return (series - s.min()) / (s.max() - s.min())

# Raw component columns
metro["company_growth"] = metro["firms_founded_yoy"]
metro["industry_diversity"] = metro["industry_count_new"].fillna(0)
metro["housing_price_growth"] = metro["fhfa_yoy"].fillna(metro["zillow_yoy"])
# Affordability: higher = better → use negative price level (lower price = higher score)
metro["housing_affordability"] = -metro["zillow_price_q"]
metro["housing_affordability"] = metro["housing_affordability"].fillna(-metro["fhfa_index"])

# Normalized [0, 1] (higher = better for all after definitions above)
for col in ["company_growth", "industry_diversity", "housing_price_growth", "housing_affordability"]:
    metro[col + "_n"] = normalize_0_1(metro[col])

# Composite: equal weight (or weight by preference)
w1, w2, w3, w4 = 0.25, 0.25, 0.25, 0.25
metro["composite_score"] = (
    w1 * metro["company_growth_n"] +
    w2 * metro["industry_diversity_n"] +
    w3 * metro["housing_price_growth_n"] +
    w4 * metro["housing_affordability_n"]
)
metro = metro.dropna(subset=["composite_score"])
print("Normalized components and composite (sample):")
metro[["metro_name", "company_growth_n", "industry_diversity_n", "housing_price_growth_n", "housing_affordability_n", "composite_score"]].head(10)

Normalized components and composite (sample):


,metro_name,company_growth_n,industry_diversity_n,housing_price_growth_n,housing_affordability_n,composite_score
0,"Abilene, TX",0.479167,0.096774,0.465499,0.780621,0.455515
1,"Akron, OH",0.497727,0.211144,0.374972,0.772253,0.464024
2,"Albany, GA",0.750000,0.129032,0.159077,0.825873,0.465995
3,"Albany, OR",0.250000,0.064516,0.637400,0.614909,0.391706
5,"Albuquerque, NM",0.273065,0.397849,0.335744,0.661950,0.417152
8,"Altoona, PA",0.125000,0.064516,0.257432,0.799961,0.311727
9,"Amarillo, TX",0.426136,0.105572,0.322251,0.769912,0.405968
10,"Ames, IA",0.484375,0.096774,0.354638,0.715746,0.412883
13,"Anchorage, AK",0.412500,0.115207,0.207012,0.519942,0.313665
14,"Ann Arbor, MI",0.414635,0.483871,0.572013,0.575449,0.511492


## Rank metros overall

Rank by composite score (higher = better "next hub + housing opportunity" under equal weights).

In [4]:
rank_overall = (
    metro.sort_values("composite_score", ascending=False)
    [["metro_name", "composite_score", "company_growth_n", "industry_diversity_n", "housing_price_growth_n", "housing_affordability_n"]]
    .reset_index(drop=True)
)
rank_overall["rank"] = rank_overall.index + 1
rank_overall = rank_overall[["rank", "metro_name", "composite_score"] + [c for c in rank_overall.columns if c not in ["rank", "metro_name", "composite_score"]]]
print(f"All metros ranked by composite score: {len(rank_overall)} metros")
rank_overall

All metros ranked by composite score: 208 metros


,rank,metro_name,composite_score,company_growth_n,industry_diversity_n,housing_price_growth_n,housing_affordability_n
0,1,"Columbus, OH",0.624295,0.216495,1.000000,0.590865,0.689820
1,2,"Fresno, CA",0.571871,0.779121,0.307692,0.599164,0.601505
2,3,"Syracuse, NY",0.565576,0.829327,0.203474,0.461486,0.768016
3,4,"Jacksonville, FL",0.565363,0.252498,0.788018,0.574070,0.646867
4,5,"Goldsboro, NC",0.557181,1.000000,0.129032,0.296785,0.802906
...,...,...,...,...,...,...,...
203,204,"Las Cruces, NM",0.302108,0.250000,0.064516,0.201584,0.692333
204,205,"Mankato, MN",0.293228,0.041667,0.129032,0.310300,0.691912
205,206,"Farmington, NM",0.288846,0.250000,0.064516,0.130335,0.710535
206,207,"Salisbury, MD",0.255835,0.250000,0.064516,0.165651,0.543174


### Column definitions (composite ranking)

**Ranking:** Metros are sorted by `composite_score` (descending), then assigned rank 1, 2, ..., N.

**Columns explained:**

- **`rank`**: Sequential rank (1 = highest composite score, N = lowest).
- **`metro_name`**: Metropolitan area name (CBSA).
- **`composite_score`**: Weighted average of 4 normalized components (equal weights: 0.25 each). Range [0, 1]; higher = better "next hub + housing opportunity".
- **`company_growth_n`**: Normalized firm formation growth. **Standardization:** Min-max to [0, 1] from raw `firms_founded_yoy` (year-over-year growth ratio). Higher = more new firms relative to other metros.
- **`industry_diversity_n`**: Normalized industry diversity. **Standardization:** Min-max to [0, 1] from raw `industry_count_new` (number of different industries among new firms). Higher = more diverse industry mix.
- **`housing_price_growth_n`**: Normalized housing appreciation. **Standardization:** Min-max to [0, 1] from raw `fhfa_yoy` (FHFA year-over-year growth). Higher = faster price appreciation.
- **`housing_affordability_n`**: Normalized affordability proxy. **Standardization:** Min-max to [0, 1] from `-zillow_price_q` (negative price level, so lower price = higher score). Higher = more affordable (lower price level). *Note: No income data available, so this is price-level based only.*


## Rank by industry (correlation-based)


In [5]:

df_corr = df.dropna(subset=["firms_founded_yoy", "fhfa_yoy"]).copy()
corr_by_metro = df_corr.groupby("cbsa_code", group_keys=False).apply(
    lambda g: g["firms_founded_yoy"].corr(g["fhfa_yoy"]) if len(g) >= 8 else np.nan,
    include_groups=False,
).reset_index()
corr_by_metro.columns = ["cbsa_code", "firm_housing_corr"]
metro_with_corr = metro.merge(corr_by_metro, on="cbsa_code", how="left")

metro_with_corr["corr_rank"] = metro_with_corr["firm_housing_corr"].rank(ascending=False, method="min", na_option="bottom")
rank_by_corr = (
    metro_with_corr.sort_values("corr_rank")
    [["metro_name", "firm_housing_corr", "industry_diversity_n", "composite_score", "corr_rank"]]
    .reset_index(drop=True)
)
rank_by_corr.insert(0, "rank", range(1, len(rank_by_corr) + 1))
# Drop duplicate corr_rank column from display if desired; keep rank as 1..N
rank_by_corr = rank_by_corr.drop(columns=["corr_rank"])
print(f"All metros in dataset ranked by firm–housing growth correlation: {len(rank_by_corr)} metros")
rank_by_corr

All metros in dataset ranked by firm–housing growth correlation: 208 metros


/Users/rishil/miniforge3/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3023: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/rishil/miniforge3/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3024: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,rank,metro_name,firm_housing_corr,industry_diversity_n,composite_score
0,1,"Monroe, MI",0.868052,0.080645,0.417069
1,2,"Wichita Falls, TX",0.804075,0.077419,0.412042
2,3,"Gettysburg, PA",0.795888,0.086022,0.350368
3,4,"Pueblo, CO",0.767791,0.073733,0.423797
4,5,"Florence, SC",0.744575,0.064516,0.308561
...,...,...,...,...,...
203,204,"Sumter, SC",NaN,0.064516,0.350358
204,205,"Goldsboro, NC",NaN,0.129032,0.557181
205,206,"Glens Falls, NY",NaN,0.064516,0.312184
206,207,"Farmington, NM",NaN,0.064516,0.288846


### Column definitions (correlation-based ranking)

**Ranking:** Metros are sorted by `firm_housing_corr` (descending), then assigned rank 1, 2, ..., N. Metros with `NaN` correlation (insufficient data) are ranked at the bottom.

**Columns explained:**

- **`rank`**: Sequential rank (1 = highest correlation, N = lowest or missing).
- **`metro_name`**: Metropolitan area name (CBSA).
- **`firm_housing_corr`**: **Pearson correlation coefficient** between `firms_founded_yoy` and `fhfa_yoy` *within each metro* over quarters (2010+). Range [-1, 1]. Higher = firm formation and housing growth move together more strongly. **Computation:** Requires ≥8 quarters with both variables non-null; otherwise `NaN`.
- **`industry_diversity_n`**: Normalized industry diversity (same as composite ranking). **Standardization:** Min-max to [0, 1] from `industry_count_new`. Higher = more diverse.
- **`composite_score`**: Composite score from section 2 (same as composite ranking). Included for reference; not used for this ranking.


## 4. Export ranked lists and expectations

Save ranked city lists for interpretability; document what each ranking means and what to expect.

In [6]:
out_dir = Path("..") / "data" / "outputs"
out_dir.mkdir(parents=True, exist_ok=True)
rank_overall.to_csv(out_dir / "metro_rank_overall_composite.csv", index=False)
rank_by_corr.to_csv(out_dir / "metro_rank_by_firm_housing_correlation.csv", index=False)
metro_with_corr.to_csv(out_dir / "metro_scores_and_correlation.csv", index=False)
print("Saved:", list(out_dir.glob("*.csv")))

Saved: [PosixPath('../data/outputs/metro_rank_by_firm_housing_correlation.csv'), PosixPath('../data/outputs/metro_scores_and_correlation.csv'), PosixPath('../data/outputs/metro_rank_overall_composite.csv')]
